# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [19]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, theta, gamma, p, seed):
    return [n, alpha, beta, theta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), theta=(0.0, 0.5), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), FloatSlider(value=1.0, description=…

### Set global variables:

In [26]:
n, alpha, beta, theta, gamma, p, seed = params.result

In [28]:
n

3

### Define particle and environment classes:

In [45]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        
    def update_position(self, value=1):
        self._position += value
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/self.alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle()
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        travel_times = [[0, None]]
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle()
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]: # changed to refer to self.n instead of n
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[self.n-1] = 0 # changed to refer to self.n instead of n
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] == 1: # changed to refer to self.n instead of n
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    particle.update_position()
                    
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format(travel_times))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [43]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, value=1):
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta, theta=theta):
        Environment.__init__(self, n, alpha, beta)
        self.theta = theta
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        self.num_roads = num_roads
        self.intersections = []
        [[self.intersections.append((x, y)) for x in self.vertical_roads] for y in self.horizontal_roads]
        self.inter_directions = {}
        for i in self.intersections:
            self.inter_directions[i] = 0
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        print(self.occupied_positions)
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/(self.alpha * sum(self.num_roads)))
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle2D(direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
        self.particles.append(particle)
        if not particle.get_direction():
            road = np.random.choice(self.horizontal_roads)
            particle.set_position([0, road])
            self.occupied_positions[0, road] = 1
        else:
            road = np.random.choice(self.vertical_roads)
            particle.set_position([road, 0])
            self.occupied_positions[road, 0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads) + self.theta * (sum(self.num_roads) ** 2)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))] + [self.theta for _ in range(sum(self.num_roads) ** 2)]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))] + ['intersection' for _ in range(sum(self.num_roads) ** 2)], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                    
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[self.n-1][road]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [self.n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[self.n-1][road] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][self.n-1]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [road, self.n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][self.n-1] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
            elif particle == 'intersection':
                i = np.random.choice(range(len(self.intersections)))
                intersection = self.intersections[i]
                self.inter_directions[intersection] = 0 if self.inter_directions[intersection] else 1
                actions += [(curr_time, 'intersection' + str(intersection), self.inter_directions[intersection])] 
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    pos = (particle_pos[0] + 1, particle_pos[1])
                    if pos in self.intersections and self.inter_directions[pos]:
                        continue
                    elif particle_pos[0] == self.n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    pos = (particle_pos[0], particle_pos[1] + 1)
                    if pos in self.intersections and not self.inter_directions[pos]:
                        continue
                    elif particle_pos[1] == self.n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Run simulator:

In [47]:
env = Env2D(num_roads=(3,3))
env.run_simulation()

[[1 1 0]
 [0 0 1]
 [1 0 0]]

 done
Summary:
Total entrances: 178
Total exits: 174

 Log:
Time: 0.1001025851029347
Particle: 0
New position: [0, 2]


Time: 0.15032239281424348
Particle: intersection(0, 1)
New position: 1


Time: 0.18478318535201105
Particle: 0
New position: [0, 3]


Time: 0.1851823472967154
Particle: 1
New position: [0, 2]


Time: 0.19525229932480925
Particle: intersection(2, 0)
New position: 1


Time: 0.22593981971209112
Particle: 1
New position: [0, 2]


Time: 0.23385642306044702
Particle: 2
New position: [2, 0]


Time: 0.24601273080110284
Particle: 2
New position: [2, 0]


Time: 0.28451093061792737
Particle: 3
New position: [0, 2]


Time: 0.4242077631067858
Particle: intersection(2, 1)
New position: 1


Time: 0.6201113934694001
Particle: intersection(1, 0)
New position: 1


Time: 0.657539561373643
Particle: intersection(1, 2)
New position: 1


Time: 0.7319389212627282
Particle: 4
New position: [1, 2]


Time: 0.7610126127228329
Particle: 5
New position: [2, 0]


Time:


Time: 9.714582006062528
Particle: intersection(2, 0)
New position: 1


Time: 9.8053317185031
Particle: intersection(1, 0)
New position: 0


Time: 9.874237661174954
Particle: intersection(0, 0)
New position: 1


Time: 9.933239683341016
Particle: intersection(2, 2)
New position: 0


Time: 10.045568040106748
Particle: intersection(0, 0)
New position: 0


Time: 10.119446557535218
Particle: intersection(1, 2)
New position: 1


Time: 10.124280080366757
Particle: 36
New position: [2, 0]


Time: 10.155965990077954
Particle: intersection(2, 0)
New position: 0


Time: 10.162059967916854
Particle: intersection(2, 1)
New position: 0


Time: 10.18838220988711
Particle: intersection(1, 1)
New position: 1


Time: 10.286675434232388
Particle: intersection(0, 1)
New position: 0


Time: 10.385968586532316
Particle: 36
New position: [3, 0]


Time: 10.540837739057173
Particle: intersection(0, 1)
New position: 1


Time: 10.578355476863322
Particle: intersection(2, 0)
New position: 1


Time: 10.64392961929

Particle: intersection(0, 2)
New position: 0


Time: 18.022760430296273
Particle: 55
New position: [2, 2]


Time: 18.17960929255
Particle: intersection(0, 1)
New position: 0


Time: 18.264940172882056
Particle: intersection(0, 1)
New position: 1


Time: 18.266250821982943
Particle: 45
New position: [3, 1]


Time: 18.364777793696565
Particle: intersection(2, 1)
New position: 1


Time: 18.40354221147479
Particle: 54
New position: [3, 0]


Time: 18.44416845365647
Particle: 56
New position: [2, 2]


Time: 18.44822783631996
Particle: intersection(1, 2)
New position: 1


Time: 18.457235767611824
Particle: 56
New position: [2, 2]


Time: 18.565020704769463
Particle: intersection(0, 2)
New position: 1


Time: 18.607210374168154
Particle: 57
New position: [0, 2]


Time: 18.63863512796163
Particle: intersection(0, 2)
New position: 0


Time: 18.68235898102575
Particle: 58
New position: [2, 1]


Time: 18.785430045918606
Particle: 56
New position: [2, 2]


Time: 18.797218851711087
Particle: interse

New position: [1, 2]


Time: 25.862081825823896
Particle: 79
New position: [1, 3]


Time: 25.901243009640055
Particle: 81
New position: [1, 2]


Time: 26.0556581192221
Particle: intersection(1, 2)
New position: 1


Time: 26.182757470701205
Particle: 80
New position: [3, 0]


Time: 26.31220630259157
Particle: intersection(1, 2)
New position: 0


Time: 26.32132691035453
Particle: 58
New position: [3, 1]


Time: 26.32565649037387
Particle: 81
New position: [1, 2]


Time: 26.45606713782405
Particle: 74
New position: [2, 1]


Time: 26.464675365839017
Particle: intersection(0, 1)
New position: 0


Time: 26.491148987371524
Particle: 81
New position: [1, 3]


Time: 26.507521764064744
Particle: intersection(0, 0)
New position: 0


Time: 26.54388922189755
Particle: 82
New position: [2, 0]


Time: 26.589499166396877
Particle: intersection(0, 1)
New position: 1


Time: 26.605470511021853
Particle: 67
New position: [1, 2]


Time: 26.639332344678568
Particle: 83
New position: [0, 2]


Time: 26.68706

New position: 1


Time: 39.49291967367758
Particle: 124
New position: [2, 2]


Time: 39.49805952869777
Particle: 122
New position: [3, 0]


Time: 39.50305841409862
Particle: intersection(1, 1)
New position: 1


Time: 39.508231519644625
Particle: intersection(1, 0)
New position: 1


Time: 39.55067807993634
Particle: intersection(2, 0)
New position: 0


Time: 39.798076548268604
Particle: 106
New position: [2, 1]


Time: 39.847017666097905
Particle: 124
New position: [2, 2]


Time: 39.873870253705036
Particle: 125
New position: [0, 2]


Time: 39.9944075630226
Particle: intersection(2, 0)
New position: 1


Time: 40.03132578689041
Particle: 124
New position: [2, 2]


Time: 40.10047833163801
Particle: 126
New position: [2, 0]


Time: 40.110966260831496
Particle: intersection(0, 1)
New position: 1


Time: 40.15267926441574
Particle: intersection(1, 1)
New position: 0


Time: 40.18357740415322
Particle: intersection(2, 0)
New position: 0


Time: 40.18469101502591
Particle: intersection(0, 0)
N

Time: 49.45921182389003
Particle: 152
New position: [0, 2]


Time: 49.49619582030132
Particle: 148
New position: [2, 2]


Time: 49.56558353882393
Particle: 148
New position: [3, 2]


Time: 49.62969102377597
Particle: 120
New position: [1, 2]


Time: 49.72114545486289
Particle: 151
New position: [3, 0]


Time: 49.76054737762429
Particle: 144
New position: [1, 2]


Time: 49.825640315661744
Particle: intersection(0, 1)
New position: 0


Time: 50.06705403338725
Particle: intersection(1, 0)
New position: 0


Time: 50.23045295604911
Particle: 153
New position: [1, 2]


Time: 50.25156509883737
Particle: 120
New position: [1, 3]


Time: 50.255581169259266
Particle: 154
New position: [2, 1]


Time: 50.32351647998992
Particle: intersection(0, 1)
New position: 1


Time: 50.33727066303454
Particle: 152
New position: [0, 3]


Time: 50.41603034859059
Particle: intersection(0, 1)
New position: 0


Time: 50.46301727215519
Particle: 154
New position: [2, 1]


Time: 50.51995249957756
Particle: intersect

In [46]:
env1 = Environment()
env1.run_simulation(time_len = 10)

[1, 1, 0]
 done
Summary:
Total entrances: 9
Total exits: 7
Travel times: [2.076159462772241, 2.0552424760807932, 3.5821019646404655, 2.7430592462237495, 0.4905228109735207, 1.3170615146344629, 1.1281960941832452]
Average travel time: 1.9131919385012115

 Log:
Time: 0.5983276403046734
Particle: 0
New position: 0


Time: 0.9820695228046944
Particle: 0
New position: 1


Time: 1.059995187276892
Particle: 0
New position: 2


Time: 1.144496882314644
Particle: 1
New position: 0


Time: 1.5748946142578464
Particle: 1
New position: 1


Time: 1.6679609741965034
Particle: 2
New position: 0


Time: 2.076159462772241
Particle: 0
New position: exit


Time: 2.5197312667048335
Particle: 1
New position: 2


Time: 3.199739358395437
Particle: 1
New position: exit


Time: 4.460987398852678
Particle: 2
New position: 1


Time: 4.583005888840831
Particle: 3
New position: 0


Time: 4.659814217769122
Particle: 2
New position: 2


Time: 5.250062938836969
Particle: 2
New position: exit


Time: 6.254520599265779
